llama3改进：
1. tokenizer
使用tiktoken， tiktoken 使用 BPE 算法，能有效处理多语言和特殊字符，减少 OOV 问题。而不是直接使用空格分词；词表更大；优化特殊token。
2. GQA
query为每个头独立生成，key和value只为部分头生成（分组生成）
在传统的多头自注意力（Multi-Head Attention）中，Query、Key、Value 都是为每个头独立生成的，计算量和显存消耗都很大。
GQA 通过减少 Key/Value 的数量，显著降低了推理和训练的资源消耗，尤其在大模型（数十亿参数）中效果明显。


In [ ]:
# Q: [batch, seq, num_heads, head_dim]
# K/V: [batch, seq, num_kv_heads, head_dim]
Q = linear_q(x)  # num_heads
K = linear_k(x)  # num_kv_heads
V = linear_v(x)  # num_kv_heads

# 将 Q 头分组，每组共享一组 K/V
Q = Q.reshape(batch, seq, num_groups, heads_per_group, head_dim)
K = K.reshape(batch, seq, num_groups, head_dim)
V = V.reshape(batch, seq, num_groups, head_dim)

# 计算注意力
attn_scores = Q @ K.transpose(-1, -2) / sqrt(head_dim)
attn_probs = softmax(attn_scores)
output = attn_probs @ V

3. RMSNorm
Layer Normalization） 是一种替代 LayerNorm 的归一化方法，常用于 Llama3 等大模型。它的主要特点是只对输入的均方根（RMS）进行归一化，不涉及均值和偏置，参数更少，计算更高效。
假设输入为 $x \in \mathbb{R}^d$，$\epsilon$ 是一个很小的常数防止除零，$\gamma$ 是可学习的缩放参数：

$$ \text{RMSNorm}(x) = \frac{x}{\text{RMS}(x) + \epsilon} \cdot \gamma $$

其中：

$$ \text{RMS}(x) = \sqrt{\frac{1}{d} \sum_{i=1}^d x_i^2} $$


In [ ]:
import torch
import torch.nn as nn

class RMSNorm(nn.Module):
    def __init__(self, dim, eps=1e-8):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def forward(self, x):
        # x: [batch, ..., dim]
        rms = x.pow(2).mean(-1, keepdim=True).sqrt()
        x_norm = x / (rms + self.eps)
        return x_norm * self.weight
"""
避免均值漂移：LayerNorm 归一化时会减去均值，可能导致数值漂移，
RMSNorm 只缩放不平移，数值更稳定。
梯度更平滑：RMSNorm 的归一化方式对梯度影响较小，训练大模型时更容易收敛，
表现出更好的稳定性。
"""

4. ROPE
RoPE 通过将每个 token 的 Query 和 Key 向量在特定维度上进行“旋转”，把位置信息直接编码进注意力计算中。
这种方式不需要额外的可学习参数，也不需要将位置编码与输入相加，而是通过数学变换实现。
对于每个 token 的向量 $x$，RoPE 将其分为偶数和奇数维度，然后用如下方式编码第 $i$ 个 token 的位置 $p$：

$$ \text{RoPE}(x, p) = x_{2k} \cdot \cos(\theta_p) + x_{2k+1} \cdot \sin(\theta_p) \ \text{RoPE}(x, p) = -x_{2k} \cdot \sin(\theta_p) + x_{2k+1} \cdot \cos(\theta_p) $$

其中 $\theta_p$ 是与位置 $p$ 和维度 $k$ 相关的角度。

5. KV cache
首次输入：模型对输入序列计算所有 Key/Value，并存入缓存。
生成新 token：只需对新 token 计算 Key/Value，然后与缓存拼接，参与注意力计算。
下次生成：继续复用缓存，直到生成结束。

6. SwiGLU
SwiGLU 结合了门控机制和 Swish 激活，具体结构如下：

$$ \text{SwiGLU}(x) = (xW_1) \odot \text{Swish}(xW_2) $$

其中：

$x$ 是输入向量
$W_1, W_2$ 是线性变换权重
$\odot$ 表示逐元素乘法
$\text{Swish}(z) = z \cdot \sigma(z)$，$\sigma$ 是 Sigmoid 函数

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SwiGLU(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(input_dim, hidden_dim)

    def forward(self, x):
        return self.linear1(x) * F.silu(self.linear2(x))